In [1]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import datetime as dt
import tensorflow as tf
from tensorflow import keras
from keras.layers import GRU, Dropout, SimpleRNN, LSTM, Dense, SimpleRNN, GRU
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import plotly as py
import plotly.io as pio

# Mute sklearn warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

from plotly.offline import init_notebook_mode

# Show charts when running kernel
init_notebook_mode(connected=True)

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore

/Users/rayhanzirvi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [48]:
# Path to your CSV file
csv_file_path = '/Users/rayhanzirvi/Desktop/acs_5yr_est_selected_economic_characteristics_2010-2022.csv'
# csv_file_path = 'math-ysics/SpringData2024/data/consume-produce_data/annual_generation_state.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# for col in df.columns:
#     print(col)

# Display the first few rows of the DataFrame
print(df.head())

# df_good = df[['State', '2021']]

# print(df_2021)



               Label (Grouping)           Category    State   Estimate  \
0             EMPLOYMENT STATUS             Header  Alabama        NaN   
1  Population 16 years and over  EMPLOYMENT STATUS  Alabama  3,714,504   
2                In labor force  EMPLOYMENT STATUS  Alabama  2,246,848   
3          Civilian labor force  EMPLOYMENT STATUS  Alabama  2,230,879   
4                      Employed  EMPLOYMENT STATUS  Alabama  2,036,867   

  Margin of Error    Percent Percent Margin of Error  Year  
0             NaN        NaN                     NaN  2010  
1          ±1,588  3,714,504                     (X)  2010  
2          ±5,576      60.5%                    ±0.1  2010  
3          ±5,531      60.1%                    ±0.1  2010  
4          ±5,904      54.8%                    ±0.2  2010  


In [49]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lstrip()

In [92]:
employed = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Employed') & (df['Category'] == 'EMPLOYMENT STATUS')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
employed = employed.drop_duplicates(subset='State', keep='first')

employed = employed[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

employed.head()

# percent is out of TOTAL POPULATION

,State,Estimate,Margin of Error,Percent,Year
90484,Alabama,"2,173,597","±8,587",54.3%,2021
90629,Alaska,"338,664","±2,500",59.2%,2021
90774,Arizona,"3,210,791","±7,197",56.8%,2021
90919,Arkansas,"1,310,863","±6,350",55.0%,2021
91064,California,"18,676,721","±21,726",59.3%,2021


In [93]:
unemployed = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Unemployed') & (df['Category'] == 'EMPLOYMENT STATUS')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
# unemployed_update = unemployed.drop_duplicates(subset='State', keep='first')
unemployed = unemployed[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

unemployed.head()

# percent is out of TOTAL POPULATION

,State,Estimate,Margin of Error,Percent,Year
90485,Alabama,"124,416","±3,443",3.1%,2021
90630,Alaska,"25,054","±1,280",4.4%,2021
90775,Arizona,"191,115","±3,619",3.4%,2021
90920,Arkansas,"73,733","±2,564",3.1%,2021
91065,California,"1,303,741","±10,259",4.1%,2021


In [94]:
unemployed['Percent'] = unemployed['Percent'].str.rstrip('%').astype('float')

unemployed.head()

,State,Estimate,Margin of Error,Percent,Year
90485,Alabama,"124,416","±3,443",3.1,2021
90630,Alaska,"25,054","±1,280",4.4,2021
90775,Arizona,"191,115","±3,619",3.4,2021
90920,Arkansas,"73,733","±2,564",3.1,2021
91065,California,"1,303,741","±10,259",4.1,2021


In [95]:
employed['Percent'] = employed['Percent'].str.rstrip('%').astype('float')

employed.head()

,State,Estimate,Margin of Error,Percent,Year
90484,Alabama,"2,173,597","±8,587",54.3,2021
90629,Alaska,"338,664","±2,500",59.2,2021
90774,Arizona,"3,210,791","±7,197",56.8,2021
90919,Arkansas,"1,310,863","±6,350",55.0,2021
91064,California,"18,676,721","±21,726",59.3,2021


In [97]:
employed.set_index('State', inplace=True)
unemployed.set_index('State', inplace=True)

In [98]:
percent_unemploy = unemployed['Percent'] / employed['Percent'] * 100

percent_unemploy.head()

State
Alabama       5.709024
Alaska        7.432432
Arizona       5.985915
Arkansas      5.636364
California    6.913997
Name: Percent, dtype: float64

In [100]:
mean_income = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Mean household income (dollars)')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
mean_income = mean_income.drop_duplicates(subset='State', keep='first')

mean_income = mean_income[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

mean_income.head()

,State,Estimate,Margin of Error,Percent,Year
90548,Alabama,"76,150",±467,(X),2021
90693,Alaska,"103,258","±1,045",(X),2021
90838,Arizona,"89,693",±503,(X),2021
90983,Arkansas,"73,346",±765,(X),2021
91128,California,"119,149",±348,(X),2021


In [102]:
median_income = df[(df['Year'] == 2021) & (df['Label (Grouping)'] == 'Median household income (dollars)')]
# Now, drop duplicates based on 'State', keeping only the first occurrence
median_income = median_income.drop_duplicates(subset='State', keep='first')

median_income = median_income[['State', 'Estimate', 'Margin of Error', 'Percent', 'Year']]

median_income.head()

,State,Estimate,Margin of Error,Percent,Year
90547,Alabama,"54,943",±377,(X),2021
90692,Alaska,"80,287","±1,113",(X),2021
90837,Arizona,"65,913",±387,(X),2021
90982,Arkansas,"52,123",±458,(X),2021
91127,California,"84,097",±236,(X),2021
